<h1>PRIMERA APROXIMACIÓN AL PROBLEMA</h1>

<h2> INTRODUCCIÓN </h2>

Para ilustrar el ejemplo se utiliza el dataset [SURVEY LUNG CANCER](https://www.kaggle.com/datasets/mysarahmadbhat/lung-cancer), cuyas características son:
 
Total no. of attributes: 16

No .of instances: 284

Attribute information:

    Gender: M(male), F(female)
    Age: Age of the patient
    Smoking: YES=2 , NO=1.
    Yellow fingers: YES=2 , NO=1.
    Anxiety: YES=2 , NO=1.
    Peer_pressure: YES=2 , NO=1.
    Chronic Disease: YES=2 , NO=1.
    Fatigue: YES=2 , NO=1.
    Allergy: YES=2 , NO=1.
    Wheezing: YES=2 , NO=1.
    Alcohol: YES=2 , NO=1.
    Coughing: YES=2 , NO=1.
    Shortness of Breath: YES=2 , NO=1.
    Swallowing Difficulty: YES=2 , NO=1.
    Chest pain: YES=2 , NO=1.
    Lung Cancer: YES , NO.

El problema que utilizaremos como ejemplo de aplicación es un análisis sencillo. Queremos contar cuántos casos de cáncer existen en función de si los individuos son fumadores o no. 


<h2> HERRAMIENTAS UTILIZADAS </h2>

La salida es un documento html exportado desde una libreta de Jupyter Notebook. El código está escrito en lenguaje Python3 y la matemática con comandos de $\LaTeX$. 

<h2> CIFRADO HOMOMÓRFICO </h2>

El uso de un sistema de cifrado homomórfico está motivado por la obtención de un algoritmo de descifrazo que defina, para una clave privada, un homomorfismo de grupos entre el espacio del mensaje cifrado ($\mathcal{C}$) y el del mensaje sin cifrar ($\mathcal{M}$): 

$$ f: \mathcal{C} \rightarrow \mathcal{M} $$

Así, podremos enviar la información cifrada al servicio Cloud, para tratarla encriptada y, posteriormente, traer al local las conclusiones para ser descifradas. Para una primera aproximación tomamos el sistema RSA simplificado. 

La clave pública es un par $(N,e)$ donde $N = p \cdot q$ es producto de dos primos $p$ y $q$ suficientemente grandes y distintos. Por otro lado, $e$ es el llamado exponente de cifrado que es un entero positivo tal que mcd $(e,\phi(N)) = 1$, donde $\phi(N) = (p-1)(q-1)$ es la función $\phi$ de Euler. 

La clave privada es el par $(N,d)$ donde $d$ es el exponente de descifrado (i.e. el inverso de $e$ módulo $\phi(n)$). De este modo se verifica que 

$$ e \cdot d \equiv 1\, (mod\, (p-1)(q-1))$$

así 

$$ \mathcal{M} = \mathcal{C} = (\mathbb{Z}_N)^* $$

que es el grupo multiplicativo de las unidades módulo n, cuyos elementos son 

$$ m \in \mathbb{Z}: 1 \leq m < N \wedge mcd\, (m,n) = 1 $$

quedando el homomorfismo de grupos 

$$ E: (\mathbb{Z}_N)^* \longrightarrow (\mathbb{Z}_N)^* $$
$$ m \mapsto m^e\, mod \, N $$

que es un isomorfismo, que permite descifrar un mensaje cifrado $c$ através de su inverso  

$$D(c) = c^d \, mod \, N$$

La utilidad de esto radica en que $((\mathbb{Z}_N)^*,\; \cdot)$ es un grupo multiplicativo, por tanto 

$$ \begin{equation} \tag{h}
E(m_1 \cdot m_2) = E(m_1) \cdot E(m_2)
\end{equation} 
$$  

<h2>APLICACIÓN AL DATASET DEL CÁNCER DE PULMÓN</h2>

En primer lugar vamos a definir dos funciones, una para encriptar un tipo int y otra para desencriptarlo. Los parámetros de entrada para la función que encripta serán: 
<ul>
<li>publicKey -> Lista que contiene: N, e </li>
<li>mensaje -> int </li>
</ul>
mientras que para la función que desencipta tendremos:
<ul>
<li>privateKey -> Lista que contiene: N, d </li>
<li>criptotexto -> int </li>
</ul>

Las funciones se muestran a continuación: 

In [11]:
def E_RSA(publicKey, mensaje): 
    n = publicKey[0]
    e = publicKey[1]
    return pow(mensaje, e, n)

def D_RSA(privateKey, criptotexto): 
    n = privateKey[0]
    d = privateKey[1]
    return pow(criptotexto, d, n)

Suponemos ahora que tenemos cifrada la columna *SMOKING* del dataset. Una primera opción que se ha trabajado es exportar el csv a un data frame, pero el tipo int64 con el que trabaja pandas colisiona con el uso del método *pow*. De momento, optaré por una solución más rudimentaria para encriptar la columna y poder ilustrar este ejemplo.

In [12]:
fumar = []
with open('survey_lung_cancer.csv', 'r') as archivo: 
    lineas = archivo.read().splitlines()
    #borro encabezado
    lineas.pop(0)
    for l in lineas: 
        linea = l.split(',')
        fumar.append(int(linea[2]))

De esta forma, hemos creado la columna *fumar* que contiene los enteros 1 si la persona no era fumadora y 2 si la persona era fumadora. Ambas entradas de tipo int. De modo que ya podemos encriptar secuencialmente la columna. 

Para ello definimos los valores que gobiernan el ejemplo: 

In [13]:
e = 2**16 + 1                       #exponente de cifrado 
p = 409                             #primo 
q = 317                             #primo
n = p * q                           #producto de dos primos distintos 

phi_n = (p-1)*(q-1)                 #cálculo de la función phi de Euler para n

pubKey = [n, e]                     #Clave publica 
priKey = [n, pow(e,-1,phi_n)]       #clave privada

Ciframos la columna *fumar*

In [14]:
fumar_encrip = []
for elemento in fumar: 
    fumar_encrip.append(E_RSA(pubKey, elemento))

Ahora vamos hacer el cálculo pedido: encontrar cuántos pacientes de cáncer de pulmón eran fumadores y cuántos no. En primer lugar, las entradas del dataset son el 1 y un número primo. Por el teorema fundamental de la aritmética sabemos que todo entero positivo mayor que 1 es un número primo o bien un único producto de números primos. De modo que el número de pacientes fumadores se puede obtener fácilmente, ya que conocemos los factores involucrados en la descomposición del producto de todos los elementos de la columna *fumar_encrip*. Sólo nos falta por conocer sus exponentes para extraer la información pedida. 

Calculamos el productorio módulo n de *fumar_encrip*

In [15]:
from math import prod
producto = (prod(fumar_encrip)) % n
producto

84172

Esta operación con los datos encriptados se habría hecho en la estructura del proveedor cloud y exportaríamos al local, únicamente, la variable *producto*. 

Una vez en local, el usuario conocería la clave privada por lo que podría desencriptarlo y *factorizarlo* para extraer la información buscada gracias a (h). 

En este caso, por ser una de las entradas un 1, el único primo es 2, por lo que buscamos su exponente y mediante la diferencia respecto al total obtenemos el número de pacientes no fumadores. La función que factorizará *producto* simplemente eleva iterativamente 2, una unidad más en cada paso, hasta obtener el valor de la variable *producto*.

In [16]:
def factP(mensaje_descifrado, n): 
    result = 0
    num = 0
    while(num != mensaje_descifrado): 
        result += 1
        num = pow(2,result,n)
        
    return result 

In [17]:
fumadores = factP(D_RSA(priKey, producto), n)
total = len(fumar)
no_fumadores = abs(total - fumadores)
print(f'El total de pacientes fumadores es: {fumadores}. \nEl total de pacientes no fumadores es: {no_fumadores}. \nDe un total de {total} pacientes.')

El total de pacientes fumadores es: 174. 
El total de pacientes no fumadores es: 135. 
De un total de 309 pacientes.


<h2>LIMITACIONES</h2>
El proceso que se acaba de describir es inútil desde el punto de vista de la seguridad. Al ser un proceso determinista, es decir, los pacientes fumadores y no fumadores se cifran siempre de la misma manera, no podemos presuponer que alguien que quiera hacer un uso fraudulento de la información desconoce los elementos involucrados en el algoritmo (*principio de Kerckhoffs*). 

Esto se puede traducir en que, en el peor caso, con un par de evaluaciones obtenemos si un paciente es o no fumador: 

In [18]:
valor = E_RSA(pubKey, 1)
print(f'Buscamos la representación de 1')
if (not E_RSA(pubKey, 1) == valor):
    E_RSA(pubKey, 2) == valor
    print("Representa: Es fumador")
else: 
    print("Representa: No es fumador")

Buscamos la representación de 1
Representa: No es fumador


La probabiliad de distinguir al fumador del no fumador es 1. El objetivo sería no ser capaces de acertar con probabilidad significativamente mayor que $\dfrac{1}{2}$. 

La definición asintótica de la probabilidad insignificante supone que, si alguien intenta distinguir los mensajes lo hace con probabiliad $\dfrac{1}{2}+\dfrac{1}{f(k)}$, donde $k$ es la longitud del módulo RSA y $f(k)$ una función polinómica, entonces el algoritmo no es seguro. 

<h2>MEJORANDO EL SISTEMA</h2>

Un algoritmo de cifrado que es homomórfico y CPA seguro es el de Paillier. Se dice que un procedimiento es CPA seguro si en tiempo polinómico un atacante no puede distinguir entre dos criptotextos partiendo de dos textos sin cifrar elegidos por él.

Las fases que gobiernan el algoritmo de Paillier son tres, a saber: 
* **Generar las claves**. Se generan dos primos aleatorios $p$ y $q$ y se obtiene la clave pública $n=pq$ y la clave privada $(n, \phi(n))$. Por simplicidad, supondremos que $p$ y $q$ tienen la misma longitud. 
* **Cifrar**. El conjunto de textos sin cifrar será $\mathbb{Z}_n$ y el de criptotextos $(\mathbb{Z}_{n^2})^*$. Para cifrar un mensaje $m \in \mathbb{Z}_n$, se elige $r \in (\mathbb{Z}_{n^2})^*$ al *azar* y se calcula el criptotexto como 

$$c = E(n, m) := ((1+n)^m \cdot r^n)\, mod \, n^2 \in (\mathbb{Z}_{n^2})^* \tag{Encriptación con Paillier} $$

* **Descifrar**. Para descrifrar $c$ con la clave privada $(n, \phi(n))$ se hace $$m = D((n, \phi(n)), c) := \dfrac{(c^{\phi(n)}\, mod\, n^2)-1}{n} \cdot \phi(n)^{-1} \, mod \, n \tag{Desencriptación con Paillier} $$ donde $((c^{\phi(n)}\, mod\, n^2)-1)/n$ se calcula en $\mathbb{Z}$.

Ahora bien, $\mathbb{Z}_n$ es aditivo, y $(\mathbb{Z}_{n^2})^*$ multiplicativo, por lo que la función de descifrado de Paillier lleva un producto de criptotextos en $(\mathbb{Z}_{n^2})^*$ a la suma de los textos sin cifrar en $\mathbb{Z}_{n}$. Sean $m_1$ y $m_2 \in \mathbb{Z}_n$, entonces $c_1 = E(n, m_1)$ y $c_2 = E(n, m_2)$ de donde  

$$D((n, \phi(n), c), (c_1 \cdot c_2)\, mod \, n^2) = (m_1 + m_2) \, mod \, n \tag{Propiedad homomórfica} $$ 

La aplicación $D: (\mathbb{Z}_{n^2})^* \rightarrow \mathbb{Z}_{n}$ inducida por el algoritmo de descifrado con una clave fija es un homomorfismo de grupos (no se puede afirmar lo mismo del de cifrado por ser probabilista). No obstante, es cierto que para cifrar la suma de varios mensajes se puede tomar como criptotexto el productorio de los criptotextos asociados a cada mensaje. 

<h2>APLICACIÓN AL DATASET DEL CÁNCER DE PULMÓN</h2>

Importamos la función $ \texttt{getprimeover(N)} $ la cual retorna un primo de N-bits usando GMP o, si no está accesible, PyCrypto; que son librerías criptográficas de Python. Además, importamos $ \texttt{random} $ para hallar $r \in (\mathbb{Z}_{n^2})^* $

In [19]:
from phe.util import getprimeover
from random import *

Generamos las claves pública y privada suponiendo que la longitud de N es de 1024. Generalmente se usan longitudes de 3072 (como hace la librería $ \texttt{Paillier} $ de $ \texttt{phe} $)

In [20]:
N_LEN = 1024
p = q = n = None
n_len = 0
while n_len != N_LEN:
    p = getprimeover(N_LEN // 2)
    q = p
    while q == p:
        q = getprimeover(N_LEN // 2)
    n = p * q
    n_len = n.bit_length()


phi_n_P = (p-1) * (q-1)
public_key_P = n 
private_key_P = [n, phi_n_P]

Implementamos un procedimiento para encriptar, según lo visto en (Encriptación con Paillier), y un procedimiento para desencriptar, según lo visto en (Desencriptación con Paillier)

In [21]:
def E_PAI(publicKey, mensaje): 
    r = randint(1, n) 
    nc = n**2
    return pow((1+n), mensaje, nc) * pow(r, n, nc)

def D_PAI(privateKey, criptotexto): 
    n = privateKey[0]
    phi = privateKey[1]
    nc = n**2

    numerador = ((pow(criptotexto, phi, nc) - 1) // n) % n
    denominador = pow(phi, -1, n)
    return (numerador * denominador) % n 


Encriptamos la columna SMOKING del dataset

In [22]:
fumar_encrip_P = []
for elemento in fumar: 
    fumar_encrip_P.append(E_PAI(public_key_P, elemento))

Calculamos 
$$ \texttt{producto\_p} = \prod_{i=0}^{i = l} c_i \, mod \, n^2 $$
donde l es la longitud de la lista *fumar_encrip_P*

In [23]:
producto_P = 1
for elemento in fumar_encrip_P:
    producto_P = (producto_P * elemento) % n**2

observamos que el elemento resultante está en $ (\mathbb{Z}_{n^2})^* $

In [24]:
print(f'producto: {len(str(producto_P))} \nn**2: {len(str(n**2))}')

producto: 616 
n**2: 616


Este valor lo traeríamos al local dentro nuestra empresa. Ahora, sabemos que para descrifrarlo es un homomorfismo según la propiedad descrita en (Propiedad homomórfica), la función de descifrado obtendrá
$$  \sum_{i=0}^l m_i \, mod \, n  $$  

dicho numero debe representar la suma de pacientes fumadores y no fumadores de la columna SMOKING del dataset

In [25]:
cyf = D_PAI(private_key_P, producto_P)
cyf

483

Dado que las entradas elegidas para representar el *si* y el *no* son 2 y 1, este camino no sirve pues no sería posible. Al ser una suma, descomponerla y obtener la información deseada no es viable. El valor obtenido 483, es coherente con la solución obtenida con RSA, la cual decía: 

$$ \texttt{El total de pacientes fumadores es: 174.} $$ 
$$ \texttt{El total de pacientes no fumadores es: 135.} $$

es decir, 

$$ 172 \cdot 2 + 135 \cdot  1 = 483 $$

como hemos obtenido. El sistema funciona como se esperaba. 

No obstante, para poder usar este método de forma eficaz, necesitamos que las entradas sean, por ejemplo, 0 y 1, de forma que la suma obtenida ya representaría el total de una de las dos opciones, siendo la otra la diferencia respecto del total de casos estudiados. 

Un posible ejemplo de aplicación sería calcular la media de edad por el genero de los pacientes. Para ello, en primer lugar vamos a crear una lista en la que convertimos M en 0 y F en 1, para poder discriminar el genero del paciente. En otra lista tendremos la edad de cada paciente. 

In [26]:
genero = []
edad = []
with open('survey_lung_cancer.csv', 'r') as archivo: 
    lineas = archivo.read().splitlines()
    #borro encabezado
    lineas.pop(0)
    for l in lineas: 
        linea = l.split(',')
        genero.append(linea[0])
        edad.append(int(linea[1]))

for i in range(len(genero)):
    if genero[i] == 'M': 
        genero[i] = 0
    else: 
        genero[i] = 1

data = list(zip(genero, edad))

Mostramos una parte de la lista data

In [27]:
print(data[10:20])

[(1, 61), (0, 72), (1, 60), (0, 58), (0, 69), (1, 48), (0, 75), (0, 57), (1, 68), (1, 61)]


Ciframos la lista data 

In [28]:
generoENCP = []
for elemento in genero: 
    generoENCP.append(E_PAI(public_key_P, elemento))

In [29]:
edadENCP = []
for elemento in edad: 
    edadENCP.append(E_PAI(public_key_P, elemento))

In [30]:
data_ENCP = list(zip(generoENCP, edadENCP))

In [31]:
def prodMod(lista, mod):
    producto_P = 1
    moD = mod**2
    for elemento in lista:
        producto_P = (producto_P * elemento) % moD
    return producto_P

In [32]:
result = (prodMod(generoENCP, n), prodMod(edadENCP, n))

In [33]:
resultDECRIP = (D_PAI(private_key_P, result[0]), (D_PAI(private_key_P, result[1])))

In [34]:
total = len(genero)
mujeres = resultDECRIP[0]
hombres = abs(mujeres - total)
media_edad = resultDECRIP[1] / total

In [35]:
import matplotlib.pyplot as plt

In [36]:
plt.bar(['Hombres', 'Mujeres'], [hombres, mujeres])
plt.show()
print(f'CONCLUSIÓN: \nLa media de edad de los pacientes es de {round(media_edad, 1)} años. \nHay un total de {total} pacientes, de los cuales {hombres} son hombres, y {mujeres} son mujeres.')

CONCLUSIÓN: 
La media de edad de los pacientes es de 62.7 años. 
Hay un total de 309 pacientes, de los cuales 162 son hombres, y 147 son mujeres.


<h2> SEGURIDAD </h2>

Si reproducimos el ataque que hicimos para RSA encontramos una diferencia sustancial: **dos textos claros no siempre tienen el mismo criptotexto**. En efecto:

In [37]:
dos = E_PAI(public_key_P, 2)
otro_dos = E_PAI(public_key_P, 2)

dos == otro_dos

False

<h2>LIMITACIONES</h2>

Podría parecer que hemos resuelto el problema. Pero nada más lejos de la realidad. Hemos impuesto que nuestro sistema de cifrado, además de homórfico, sea CPA seguro. Pero se conocen formas de atacar a estos sistemas permitiendo conocer, en nuestro caso, si un paciente en concreto es o no fumador. Este tipo de ataques se conocen como ataques CCA. Como en el caso CPA se elegirán dos mensajes, y uno de ellos (al azar) es cifrado, y el criptotexto es entregado al atacante que intenta descifrar a cuál corresponde. Además, el atacante puede adaptar el ataque según las respuesta que obtenga. Veamos que ocurre con nuestro ejemplo:

En primer lugar ciframos el 1

In [38]:
uno = E_PAI(public_key_P, 1)
uno 

906477654063508770025018307487212423291485346022284055619553315578955141943449916252390206000033891859343317996511076989681241974933846723483339238452202918134461188091524642385463917826480064502922304478732758951887341426886306268465618031436961858027158506232311312284073295677310492804486129692004003650127998150947732123079513991641540078825040146422549377540456713152113215499116899360476803724575432532264906381581207763668310878165512694514040365671511617557231010979864647472318544501959922560082646122540735796679840759440085632548925701796281677849116633033437353218811205128991709513766733437728971962669665827873902915721228206972571489013568232096867625372097229237656373728699133304677921055153822732672498784466031120225723256478033433708456280832485793043635139014165657613681480516145124478905088754412218784674854058297377037605970659796283231705708702466683258624698860539289530723392494920521371011441730

El atacante, puede utilizar la propiedad homomórfica. Sea c el primer elemento de $ \texttt{fumar\_encrip\_P} $ 

$$ \texttt{c1} := (\texttt{uno} \cdot \texttt{c}) \, mod \, n^2 $$

y además, sabe que 

$$ \texttt{D\_PAI(private\_key\_p, uno) = 1} $$

In [39]:
c1 = (uno * fumar_encrip_P[0]) % (n**2) 
D_PAI(private_key_P, c1) - 1

1

ergo el primer paciente, no era fumador. 

"Un esquema criptográfico se dice no maleable si la probabilidad de que el adversario encuentre dichos textos [...] no es significativamente mayor que dicha probabilidad para el cifrado de una cadena de bits $ x $ elegida al azar." (González Vasco, 2003, p.26)

Por tanto decimos que el método de Paillier es maleable. Un atacante puede modificar el criptotexto del ataque y obtener otro criptotexto relacionado de forma que su texto claro esté relacionado con el texto claro que se quiere descifrar. 

<h2> OTROS SISTEMAS HOMOMÓRFICOS </h2>

Hemos estudiado dos casos de sistemas de encriptación con la propiedad de ser homomórficos. En una primera aproximación, nos encontramos un tipo de brecha de seguridad debido a la cual un atacante experto puede descifrar como almacenamos la información; posteriormente tratamos de solucionar esta situación introduciendo un sistema de cifrado que, además de homomórfico, impone la condición de que dos textos claros corresponden a dos criptotextos diferentes, pero de nuevo hemos encontrado una brecha de seguridad. Existen otros sistemas de cifrado como El Gamal, que son homomórficos y CPA seguros [David Wagner, Lecture 19, 2006, Berkeley]. No obstante, la maleabilidad deja de nuevo patente que no podemos confiar en este sistema para abordar el problema planteado.  

Otra brecha de seguridad, que no hemos tratado, sería la de utilizar una misma clave para todos los datos que han de ser cifrados. Además, si tenemos un dataset realista, la cantidad de datos cifrada generaría un campo de prueba y error inasumible. Desde este punto de vista, sería necesario encontrar sistemas que permitan minimizar la información cifrada con una sola clave. Existen soluciones fuera del ámbito del problema como la implantación de una capa adicional de cifrado en la propia comunicación. También se podría hacer análisis previo, que permita saber que datos se cruzaran para cifrar el dataset por bloques con diferentes claves. Existen otras soluciones, para explorar desde el punto de vista matemático, como la privacidad diferencial.

<h3> ESTADO DEL ARTE </h3>  

Este problema se remonta a 1978 cuando Rivest, Adleman y Dertouzos [Rivest, Adleman & Dertouzos, 1978,  On data banks and privacy homomorphisms. Foundations of secure computation, 169-180.] propusieron el problema de la existencia de un sistema de cifrado **completamente homomórfico**. Un sistema de cifrado completamente homomórfico soporta tanto la suma como el producto preservando así la estructura en anillo de los textos origen.  

Si este tipo sistema existiera permitiría la construcción de programas que trabajen con datos cifrados cuya salida esté también cifrada salvando todas las brechas que hemos comentado anteriormente. Como estos programas no descifran los datos se pueden ejecutar en la empresa que oferta el servicio Cloud.  

Pese a sus bondades, el cifrado completamente homomórfico aún es un reto en la práctica empresarial. Una de las mejores aproximaciones que existen en la actualidad es el sistema criptográfico Boneh-Goh-Nissim, que permitía la evaluación de un número ilimitado de operaciones de suma, pero a lo sumo una multiplicación. 

<h3> ¿EXISTE UN CIFRADO COMPLETAMENTE HOMOMÓRFICO? </h3>

La respuesta a la pregunta es: sí. Desde el punto de vista puramente matemático el problema fue resuelto por Craig Gentry mientras acababa su tesis en Stanford y trabajaba en IBM (2009). Gentry propone el primer sistema de cifrado completamente homomórfico usando la teoría de retículos. Este sistema es CPA seguro y eficiente, pues sus algoritmos son de tiempo polinómico. No obstante, aún queda pendiente conseguir una versión más eficiente para que pueda ser aplicada en la práctica. En este sentido, las empresas utilizan sistemas *algo homomórficos* que pueden calcular muchas sumas, pero sólo un pequeño número de multiplicaciones.  

<h2> SOLUCIÓN FINAL </h2> 

La diferencia entre el uso de un sistema parcialmente homomórfico (*Somewhat Homomorphic Encryption*) (SHE) y uno completamente homomórfico (*Fully Homomorphic Encryption*) (FHE), es que el segundo permite un número de operaciones no acotado. De hecho, es posible, partiendo de un sistema SHE obtener un sistema FHE. Gentry introdujo este camino (computacionalmente pesado) denominado *bootstrapping*. 

<h3> UN ESQUEMA SHE </h3>

El trabajo en este tipo de sistemas se hace en un anillo de polinomios de la forma $ \mathbb{Z}[T]/(f) $. En el que se definen operaciones de suma y multiplicación. Estas operaciones se pueden implementar utilizando la librería $ \texttt{numpy}$, en concreto la clase $ \texttt{numpy.polynomial} $. 




<h3> Fan-Vercauteren </h3>

Sea $ n $ una potencia de dos. Sean t, q enteros positivos. Tomando $ \Delta = \lfloor q / t \rfloor $ . El espacio de textos claros corresponde a $ R_t = \mathbb{Z}_t [x]/(x^n + 1) $ y el de criptotextos a $ R_q = \mathbb{Z}_q [x]/(x^n + 1) $. Denotaremos $ R = \mathbb{Z} [x]/(x^n + 1) $.

<ul>

<li> 

Generación de clave privada: es un polinomio en $ R $ con coeficientes en $ \{ 0, 1 \} $ </li>

<li> 

Generación de clave pública: tomando $ a $ en $ R_q $ y un error $ e $ en función de una cierta distribución $ \chi $ sobre $ R $ así 

$$ pubKey = ([-(a \cdot s + e)]_q, a) \in R_q \times R_q $$ 


</li>

<li>

Encriptación: sea el texto claro $ m \in R_t $ y $ pk = (pk_0, pk_1) $ entonces $$  \texttt{Enc}(pk, m) = ([pk_o \cdot u  + e_1 + \Delta \cdot m]_q, [pk_1 \cdot u + e_2]_q) \in R_q \times R_q $$ 
Debido al problema del anillo con aprendizaje de errores queda asegurado que no se revela ninguna información del texto plano. 

</li>

<li>

Desencriptación: dado un criptotexto $ ct = \texttt{Enc}(pk, m) = (ct_0, ct_1) $ se desencripta usando la clave pública $ sk = s $ según 

$$ \texttt{Dec}(sk, ct) = \left[ \left \lfloor  \dfrac{t \cdot [ct_0 + ct_1 \cdot s]_q }{q} \right \rfloor  \right]_t \in R_t $$

</li>

</ul>

 

<h3> OPERACIONES HOMOMÓRFICAS </h3>

Suma: 
$$ [ct(s)]_q + [ct'(s)]_q = \Delta \cdot (m + m') + v + v' $$

$$ [ct(s) + ct'(s)]_q = \Delta \cdot [m + m']_t + d \, mod \, q $$

donde $ d = v + v' - (q - \Delta \cdot t) \cdot \varepsilon $.

El producto requiere de una técnica intermedia llamada *relinearization*.

<h3> IMPLEMENTACIÓN </h3>

Estas técnicas están implementadas en [Mădălina Bolboceanu, Miruna Roșca, Radu Țițiu](https://github.com/bit-ml/he-scheme). No obstante, una implementación más profesional la podemos encontrar en [SEAL](https://github.com/Microsoft/SEAL). 



<h3> CONCLUSIÓN FINAL DEL TRABAJO </h3>

Es emocionante enfrentarse a problemas reales utilizando las matemáticas. El camino inicial me llevó a un planteamiento totalmente erróneo. Imaginaba textos cifrados que pudiera filtrar buscando con un *if*. Las matemáticas que fui descubriendo no sólo me sacaron de mi error, sino que me hizo comprender mejor el problema. Sin querer, he construido una solución de menos a más, lo que me ha hecho comprender la importancia de estructurar desde el pensamiento matemático. Ha sido un camino de aprendizaje desde el primer momento que me senté a buscar información. Me habría gustado contar con un poco más de tiempo para meterme de lleno con las ideas de Gentry, los algoritmos algo homomórficos o la privacidad diferencial.   

Me apasiona la tecnología, por eso elegí hacer la especialidad de computación. De entre los problemas planteados por las diferentes empresas este me llamó la atención desde el primero momento, todos los demás planteaban problemas más cerrados, concretos. Si me metí hacer matemáticas es porque me gusta enfrentarme a problemas que requieren procesos de aprendizaje constante, conocer nuevas ramas de las matemáticas, conectarlas con cosas tangibles. Finalmente, debo admitir, que ver un dataset con datos de la rama sanitaria despertó mi curiosidad. Siempre me ha llamado mucho la atención el impacto de la matemática en la sanidad.  

<h3> DATAFRAMES Y NUMPY </h3>

Uno de los problemas que me encontré fue la imposibilidad del uso del tipo int64 con la función predefinida pow. 

Una solución que he investigado es la siguiente: 

In [40]:
import pandas as pd 
import numpy as np

In [41]:
def multiplicar_mod(a, b, N):    # 0 <= a,b < N, N >= 1
   c = a * b
   c %= N
   return np.int64(c)

def potencia_mod(a, k, N):             # 0 <= a < N, k >= 0, N >= 2
    if k == 0:                         # caso base (k = 0)
        r = 1                          # convencion: 0^0 = 1
    elif k % 2 == 0:                   # k es par (k > 0)
        r = potencia_mod(a, k//2, N)
        r = multiplicar_mod(r, r, N)
    else:                              # k es impar (k > 0)
        r = potencia_mod(a, k-1, N)
        r = multiplicar_mod(a, r, N)
    return np.int64(r)

In [42]:
def E_RSA(publicKey, mensaje): 
    n = publicKey[0]
    e = publicKey[1]
    return potencia_mod(mensaje, e, n)

def D_RSA(privateKey, criptotexto): 
    n = privateKey[0]
    d = privateKey[1]
    return potencia_mod(criptotexto, d, n)

def factP(mensaje_descifrado, n): 
    result = 0
    num = 0
    while(num != mensaje_descifrado): 
        result += 1
        num = potencia_mod(2,result,n)
    return result 

De esta forma he conseguido salvar el problema de tipos. No obstante, se genera un problema que no me ha dado tiempo a solucionar con el uso del operador modular de Python. 

<h3>BIBLIOGRAFÍA</h3>

González Vasco, M. I. (2003). Criptosistemas basados en Teoría de grupos.

Milanov, E. (2009). The RSA algorithm. RSA laboratories, 1-11.

Pardo, J. L. G. (2012). Cifrado homomórfico: ejemplos y aplicaciones. Gaceta de la Real Sociedad Matematica Española, 15(4), 697-712.

Mădălina Bolboceanu, Miruna Roșca, Radu Țițiu (16 de Noviembre 2020) Homomorphic Encryption: a Toy Implementation in Python. https://bit-ml.github.io/blog/post/homomorphic-encryption-toy-implementation-in-python/